In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
from sklearn.linear_model import LinearRegression

In [60]:
def name_changer(st) -> str:
    st = st.split('\\')  # C:\Users\user\!BMM\Digital_Breakthrough\NPO_dataset
    st = '/'.join(st)
    return st

def file_scanner(path:str) -> list:
    path = name_changer(path)
    names = os.listdir(path)
    names = names[:]
    def file_loader():
        data = []
        for name in names:
            im_frame = pd.read_csv(path  + "/" + name)
            typ_data = im_frame
            typ_data = typ_data.interpolate()   ##Дозаполнение таблицы и удаление строчек, где дозаполнение невозможно
            typ_data = typ_data.dropna()
            typ_data = typ_data.reset_index(drop=True)
            data.append(typ_data)    
        return data
    data = file_loader()
    return data
def to_ticks(time_raw : str):
    time_raw = time_raw.split()[0]
    t0 = datetime.fromisocalendar(1993, 1, 7)
    time = datetime.strptime(time_raw, '%Y-%m-%d')
    ticks = (time - t0).total_seconds()
    return ticks

In [89]:
Path = r"C:\Users\user\!BMM\Digital_Breakthrough\NPO_dataset"  #путь с папкой, где лежат 3 файла данных
print(os.listdir(Path))     # проверьте что поярдкок 3х переменных ниже совпадает с порялдком считанным программой
data_files = file_scanner(Path)
trn = data_files[0]


['npo_trnsctns.csv', 'out_time.csv']


In [91]:
# cl_info = data_files[0] #clients
# acc = data_files[1] #bank accounts        #npo_sum   npo_operation_date** npo_operation_group
# trn = data_files[2] #transactions 
# trn[]
# trn = trn.rename(columns={"628548768000000000": "time", " 20.32": "money", " 0":'type'}) #time 86400 * 10**7  ##kvartal = 91day
trn = trn.drop(['slctn_nmbr', 'npo_accnt_id'], axis=1 )
trn = trn.rename(columns={"npo_operation_date": "time", "npo_sum": "money", "npo_operation_group":'type'}) #переименуюм файлы 
trn[['time', 'money', 'type']]
trn['time'] = trn['time'].map(to_ticks)  # преобразуем время в тики, чтобы можно было удобно сортировать
trn.sort_values('time')
trn['time'] = trn['time'].map(lambda x: (x //(86400 * 10**7)))  # преобразуем время в дни, чтобы можно было удобно сортировать

In [88]:
def Dataset(trn) -> list:
    try:          # попытка закэшировать данные
        a = np.load('dataset.npy')
        return a
    except FileNotFoundError:
        pass
    data_set = []
    income, outcome = 0,0
    kvartal = 91
    time = 0
    now_time = trn['time'][-2:-1].tolist()[0]
    for idx ,row in trn.iterrows():
        if row[0] > kvartal:        #проверка периода времени, если выходит за рамки квартала - выходим из цикла
            cashflow = income - outcome
            print(time/now_time, time, now_time)
            data_set.append((income, outcome, cashflow))
            kvartal += 91
            income, outcome = 0,0       
        money, opt, time = row[1], row[2], row[0]
        if money > 0 and opt == 0: income += money
        if money < 0 and opt == 0: outcome += money
        elif money > 0 and opt == 1: income += money
        np.save('dataset.npy', data_set)
        return data_set
inc, out, cash = [], [],[]
for x,y,z in Dataset(trn):
    inc.append(x)
    out.append(y)
    cash.append(z)
inc = np.array(inc)
out = np.array(out)
cash = np.array(cash)
inc = inc[1:]
out = out[1:]
cash = cash[:-1]

array([2.32223160e+05, 5.09977460e+05, 5.27645200e+05, 6.01050450e+05,
       9.65379970e+05, 1.79319670e+05, 1.12721790e+05, 4.96216900e+04,
       4.31080900e+04, 6.25259610e+05, 2.82196420e+05, 3.32874300e+05,
       4.92457610e+05, 5.11540550e+05, 4.45303220e+05, 4.96646280e+05,
       6.67924030e+05, 7.74160380e+05, 1.36314446e+06, 2.79237150e+05,
       2.04892490e+05, 2.65507451e+06, 1.83031463e+06, 1.10722872e+06,
       5.96594723e+06, 1.20713480e+06, 1.13162060e+06, 9.34938940e+05,
       4.74506198e+06, 1.29633118e+07, 1.85322878e+06, 1.03723015e+07,
       2.98081935e+06, 2.82392929e+06, 1.58994919e+07, 3.41410877e+06,
       1.05308950e+07, 2.29995801e+06, 5.29928864e+07, 3.25656225e+06,
       4.29214720e+07, 8.52435610e+05, 9.22786125e+07, 2.51697695e+07,
       1.05980168e+07, 1.46473147e+07, 2.89261573e+08, 2.65877772e+07,
       3.46692424e+07, 4.38734454e+07, 5.00444936e+08, 5.95701732e+07,
       7.26250717e+07, 7.87190083e+07, 2.55825047e+08, 8.89143762e+07,
      

In [59]:
# data_set = []
# income, outcome = 0,0
# kvartal = 91
# time = 0
# now_time = trn['time'][-2:-1].tolist()[0]
# for idx ,row in trn.iterrows():
#     if row[0] > kvartal:        #проверка периода времени, если выходит за рамки квартала - выходим из цикла
#         cashflow = income - outcome
#         print(time/now_time, time, now_time)
#         data_set.append((income, outcome, cashflow))
#         kvartal += 91
#         income, outcome = 0,0       
#     money, opt, time = row[1], row[2], row[0]
#     if money > 0 and opt == 0: income += money
#     if money < 0 and opt == 0: outcome += money
#     elif money > 0 and opt == 1: income += money


0.008499906594433029 91.0 10706
0.016999813188866057 182.0 10706
0.025499719783299086 273.0 10706
0.033999626377732114 364.0 10706
0.04249953297216514 455.0 10706
0.05090603399962638 545.0 10706
0.0594993461610312 637.0 10706
0.06771903605454885 725.0 10706
0.07640575378292547 818.0 10706
0.08499906594433028 910.0 10706
0.09349897253876331 1001.0 10706
0.10181206799925276 1090.0 10706
0.11040538016065758 1182.0 10706
0.1189986923220624 1274.0 10706
0.12749859891649543 1365.0 10706
0.13581169437698487 1454.0 10706
0.1443116009714179 1545.0 10706
0.15271810199887914 1635.0 10706
0.16149822529422753 1729.0 10706
0.16981132075471697 1818.0 10706
0.17831122734915 1909.0 10706
0.18671772837661124 1999.0 10706
0.19540444610498786 2092.0 10706
0.2039977582663927 2184.0 10706
0.21231085372688213 2273.0 10706
0.22081076032131516 2364.0 10706
0.22949747804969176 2457.0 10706
0.2378105735101812 2546.0 10706
0.24649729123855782 2639.0 10706
0.25499719783299085 2730.0 10706
0.26349710442742386 2821.

In [ ]:
# inc, out, cash = [], [],[]
# for x,y,z in data_set:
#     inc.append(x)
#     out.append(y)
#     cash.append(z)
# inc = np.array(inc)
# out = np.array(out)
# cash = np.array(cash)
# inc[1:]
# out[1:]
# cash[:-1]
# # out

In [87]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import numpy as np

# предположим, у вас есть следующие данные
income = np.array(inc)
outcome = np.array(out)
cashflow = cash

# преобразуем данные в нужный формат
X = np.vstack((income, outcome)).T
y = cashflow
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# создаем и обучаем модель
model = LinearRegression()
model.fit(X_train, y_train)
# Предсказание на тестовой выборке
y_pred = model.predict(X_test)

# Вычисление метрики MSE
mse = mean_squared_error(y_test, y_pred)
print(f'метрика MSE: {mse}')  #чем меньше значение метрики, тем лучше
print(f'Cashflow на будующий квартал {model.predict(X_test[-2:-1])}')  # кэшфлоу на убующий квартал
# теперь вы можете использовать модель для предсказания cashflow

MSE: 4.3915488529617305e-15
Cashflow на будующий квартал [1.74505376e+08]


In [ ]:
# print(model.predict(your_data))  # your data = ваши предобработтанные данные 

In [ ]:
# min(trn['time'])             ## DAtasset creating
# trn.sort_values('time')
# data_set = []
# now_time = 400#trn['time'][-2:-1].tolist()[0]
# for time in range(0, now_time, 15):  
#     sv_idx = time               #ВОзвращение счётчика начала поиска в момент вермени
#     kvartal = time + 91         #граница хода цикла
#     income, outcome = 0,0       #обнуление прибыли и издержек
#     for idx ,row in trn[sv_idx:].iterrows():
#         if row[0] > kvartal:        #проверка периода времени, если выходит за рамки квартала - выходим из цикла
#             break            
#         sv_idx = idx
#         money, opt = row[1], row[2]
#         if money > 0 and opt == 0: income += money
#         if money < 0 and opt == 0: outcome += money
#         elif money > 0 and opt == 1: income += money
#     cashflow = income - outcome
#     print(time/now_time, time, now_time)
#     print(sv_idx)
#     data_set.append((income, outcome, cashflow))